<a href="https://colab.research.google.com/github/bcataldo3/plantilla_curso_python/blob/main/docs/Clase_05 - Modelo K-Medias Metricas Evaluacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Métricas de Evaluación
Intro a la clase

## Minimizar la Inercia

Uno de los mayores inconvenientes de K-medias y algoritmos similares es la solicitud explícita del número de agrupaciones. A veces, esta información está impuesta por restricciones externas (por ejemplo, en el caso de estudio, solo hay dos posibles diagnósticos), pero en muchos casos (cuando se necesita un análisis exploratorio), el científico de datos debe verificar diferentes configuraciones y evaluarlas. La forma más sencilla de evaluar el rendimiento de K-medias y elegir un número apropiado de clústeres se basa en la comparación de diferentes inercias finales. Para explicar esto mediremos las inercias de un ejemplo generado a partir de 12 globos Gausianos muy compactos generados con la funcion de ```scikit-learn make_blobs()```

Comenzamos creando la data y mostrando resultados

<img src="https://drive.google.com/uc?export=view&id=1vuLtM8DQhLmKcN6C3-j9DknugLiwDwqt" width = "800" align="center"/>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
X, Y = make_blobs(n_samples=2000, n_features=2, centers=12,cluster_std=0.05, center_box=[-5, 5], random_state=100)
# Show the blobs
sns.set()
fig, ax = plt.subplots(figsize=(12, 8))
for i in range(12):
    ax.scatter(X[Y == i, 0], X[Y == i, 1], label='Blob {}'.format(i + 1))
ax.set_xlabel(r'$x_0$')
ax.set_ylabel(r'$x_1$')
ax.legend()
plt.show()

Ahora mostramos la grafica de la inercia para distintos números de agrupaciones y el código correspondiente.

<img src="https://drive.google.com/uc?export=view&id=1_HtS5eeEoJd6jhVeScIHm-fHtXMcI_tk" width = "800" align="center"/>

In [ ]:
# Compute the inertia
inertias = []
for i in range(2, 21):
    km = KMeans(n_clusters=i, max_iter=1000, random_state=1000)
    km.fit(X)
    inertias.append(km.inertia_)
# Show the plot inertia vs. no. clusters
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(np.arange(2, 21, 1), inertias)
ax.set_xlabel('Number of clusters', fontsize=14)
ax.set_ylabel('Inertia', fontsize=14)
ax.set_xticks(np.arange(2, 21, 1))
plt.show()

Algunos comentarios del proceso:
* El gráfico anterior muestra un comportamiento común. Cuando el número de clústeres es muy pequeño, la densidad es proporcionalmente baja, por lo tanto, la cohesión es baja y, como resultado, la inercia es alta. Aumentar el número de clústeres obliga al modelo a crear grupos más cohesionados y la inercia comienza a disminuir abruptamente. 
* La regla heurística genérica (cuando no hay restricciones externas) es elegir el número de clústeres correspondiente al punto que separa la región de alta variación de la casi plana. De esta manera, nos aseguramos de que todos los clústeres hayan alcanzado su máxima cohesión sin fragmentación interna. 
* Por supuesto, en este caso, si hubiéramos seleccionado K=15, nueve conjuntos habrían sido asignados a diferentes clústeres, mientras que los otros tres se habrían dividido en dos partes. Obviamente, al dividir una región de alta densidad, la inercia sigue siendo baja, pero el principio de separación máxima ya no se sigue.

### Caso de Estudio: Minimizando la Inercia
Ahora revisaremos la inercia de nuestro caso de estudio para diferentes niveles de conglomerados (de 2 a 50)

In [ ]:
# Compute the inertia
inertias = []

for i in range(2, 51):
    km = KMeans(n_clusters=i, max_iter=1000, random_state=1000)
    km.fit(cdf)
    inertias.append(km.inertia_)

# Show the plot inertia vs. no. clusters
fig, ax = plt.subplots(figsize=(18, 8))

ax.plot(np.arange(2, 51, 1), inertias)
ax.set_xlabel('Number of clusters', fontsize=14)
ax.set_ylabel('Inertia', fontsize=14)
ax.set_xticks(np.arange(2, 51, 2))

plt.show()



En este caso, la verdad fundamental sugiere que deberíamos agrupar en dos grupos correspondientes a los diagnósticos. Sin embargo, el gráfico muestra un descenso drástico que termina en K=8 y continúa con una pendiente más baja hasta aproximadamente K=40. Durante el análisis preliminar, hemos observado que la proyección bidimensional está compuesta por muchos conjuntos aislados que comparten el mismo diagnóstico. Por lo tanto, podríamos decidir emplear, por ejemplo, K=8 y analizar las características correspondientes a cada clúster. Dado que esto no es una tarea de clasificación, la verdad fundamental puede usarse como la principal referencia, pero un análisis exploratorio correcto puede intentar comprender la composición de las subestructuras para proporcionar detalles adicionales a los técnicos (por ejemplo, médicos).
Ahora realizaremos un agrupamiento K-means con ocho clústeres en el conjunto de datos de Breast Cancer Wisconsin para describir la estructura de dos grupos de muestra, de la siguiente manera:







In [ ]:
km = KMeans(n_clusters=8, max_iter=1000, random_state=1000)
Y_pred = km.fit_predict(cdf)

df_km = pd.DataFrame(Y_pred, columns=['prediction'], index=cdf.index)
kmdff = pd.concat([dff, df_km], axis=1)

# Show the result
fig, ax = plt.subplots(figsize=(18, 11))

with sns.plotting_context("notebook", font_scale=1.5):
    sns.scatterplot(x='x',
                    y='y',
                    hue='prediction',
                    size='area_mean',
                    style='diagnosis',
                    sizes=(30, 400),
                    palette=sns.color_palette("husl", 8),
                    data=kmdff,
                    ax=ax)

ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')

plt.show()

Ahora, consideremos el intervalo $-25 < x < 30$ y $-60 < y <-40$ y revisemos estadística descriptiva sencilla de algunos de sus atributos.

In [ ]:
sdff = dff[(dff.x > -25.0) & (dff.x < 30.0) & (dff.y > -60.0) & (dff.y < -40.0)]
print(sdff[['perimeter_mean', 'area_mean', 'smoothness_mean','concavity_mean', 'symmetry_mean']].describe())

Desde la verdad fundamental, sabemos que todas estas muestras son malignas, pero podemos intentar determinar una regla. La relación entre el área media y el perímetro medio es aproximadamente de 9.23 y las desviaciones estándar relativas son muy pequeñas en comparación con las medias. Esto significa que estas muestras representan tumores extendidos en un rango muy estrecho. Además, tanto la concavidad media como la simetría media son mayores que los valores generales. Por lo tanto (sin la presunción de un análisis científicamente razonable), podemos concluir que las muestras asignadas a estos grupos representan tumores muy graves que han alcanzado una etapa avanzada. Ahora revisemos la siguiente área $x> -10$ y $ 20 < y
< 50$

In [ ]:
sdff = dff[(dff.x > -10.0) & (dff.y > 20.0) & (dff.y < 50.0)]
print(sdff[['perimeter_mean', 'area_mean', 'smoothness_mean','concavity_mean', 'symmetry_mean']].describe())

En este caso, la relación entre el área media y el perímetro medio es aproximadamente de 4.89, pero el área media tiene una desviación estándar más grande (de hecho, su valor máximo es de alrededor de 410). La concavidad media es extremadamente pequeña en comparación con la anterior (incluso con aproximadamente la misma desviación estándar), mientras que la simetría media es casi equivalente. A partir de este breve análisis, podemos deducir que la simetría media no es una característica discriminante, mientras que una relación entre el área media y el perímetro medio menor que 5.42 (considerando los valores máximos) con una concavidad media menor o igual a 0.04 debería garantizar un resultado benigno. Dado que la concavidad media puede alcanzar un valor máximo muy grande (mayor que el asociado con muestras malignas), es necesario considerar también las otras características para decidir si su valor debe ser considerado como una alarma. Sin embargo, podemos concluir diciendo que todas las muestras pertenecientes a estos grupos son benignas con una probabilidad de error insignificante

Es importante recalcar que esto es más un ejercicio que un análisis real y, en tales situaciones, la tarea principal del científico de datos es recopilar piezas de información contextual que puedan respaldar las conclusiones. Incluso en presencia de la verdad fundamental, este proceso de validación es siempre obligatorio porque la complejidad de las causas subyacentes puede llevar a afirmaciones y reglas completamente erróneas

## Coeficiente de Silhouette
El método más común para evaluar el rendimiento de un algoritmo de agrupamiento sin conocimiento de la verdad fundamental es el puntaje de silueta. Proporciona tanto un índice por muestra como una representación gráfica global que muestra el nivel de coherencia interna y separación de los grupos. Para calcular el puntaje, necesitamos introducir dos medidas auxiliares. La primera es la distancia intra-cluster promedio de una muestra $x_i ∈ K_j$ asumiendo la cardinalidad de $|K_j|=n(j)$:

$sfjkask$

Para K-means, se asume que la distancia es euclidiana, pero no hay limitaciones específicas. Por supuesto, d(•) debe ser la misma función de distancia empleada en el procedimiento de agrupamiento. Dada una muestra xi ∈ Kj, vamos a denotar el clúster más cercano como Kc. De esta manera, también podemos definir la distancia más pequeña al clúster más cercano (como la distancia promedio al clúster más cercano)

$scsakfmascfas$

Con estas dos medidas, podemos definir el coeficiente de Silhouette para cada $x_i \in X$

$dvsdv dslv msdlds$

El puntaje s(•) ∈ (-1, 1). Cuando s(•) → -1, significa que b(•) << a(•), por lo tanto, la muestra xi ∈ Kj está más cerca del clúster más cercano Kc que de las otras muestras asignadas a Kj. Esta condición indica una asignación incorrecta. Por otro lado, cuando s(•) → 1, b(•) >> a(•), por lo que la muestra xi está mucho más cerca de sus vecinos (pertenecientes al mismo clúster) que de cualquier otro punto asignado al clúster más cercano. Claramente, esta es una condición óptima y la referencia a emplear al ajustar un algoritmo. Sin embargo, como este índice no es global, es útil introducir gráficos de silueta, que muestran los puntajes obtenidos por cada muestra, agrupados por clúster y ordenados de manera descendente.

Consideremos gráficos de Silhouette para el conjunto de datos de cancer de pecho para $K={2, 4, 6, 8}$

In [ ]:
nb_clusters = [2, 4, 6, 8]
mapping = [(0, 0), (0, 1), (1, 0), (1, 1)]

# Show the silhouette plots
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

for i, n in enumerate(nb_clusters):
    km = KMeans(n_clusters=n, random_state=1000)
    Y = km.fit_predict(cdf)
    df_km = pd.DataFrame(Y_pred, columns=['prediction'], index=cdf.index)
    kmdff = pd.concat([dff, df_km], axis=1)

    silhouette_values = silhouette_samples(cdf, kmdff['prediction'])

    ax[mapping[i]].set_xticks([-0.15, 0.0, 0.25, 0.5, 0.75, 1.0])
    ax[mapping[i]].set_yticks([])
    ax[mapping[i]].set_title('%d clusters' % n)
    ax[mapping[i]].set_xlim([-0.15, 1])
    y_lower = 20

    for t in range(n):
        ct_values = silhouette_values[Y == t]
        ct_values.sort()

        y_upper = y_lower + ct_values.shape[0]

        color = cm.Accent(float(t) / n)
        ax[mapping[i]].fill_betweenx(np.arange(y_lower, y_upper), 0, ct_values, facecolor=color, edgecolor=color)

        y_lower = y_upper + 20

    plt.show()

El primer diagrama muestra el agrupamiento natural con K=2. La primera silueta es muy marcada, lo que indica que la distancia inter-cluster promedio tiene una gran variación. Además, un clúster tiene muchas más asignaciones que el otro (aunque es menos marcado). A partir de la descripción del conjunto de datos, sabemos que las dos clases están desequilibradas (357 benignas frente a 212 malignas), por lo tanto, la asimetría está parcialmente justificada. Sin embargo, en general, cuando los conjuntos de datos están equilibrados, un buen gráfico de silueta se caracteriza por tener clústeres homogéneos con una silueta redondeada que debería estar cerca de 1.0. De hecho, cuando la forma es similar a un cigarro largo, significa que las distancias intra-cluster están muy cerca de su promedio (alta cohesión) y hay una clara separación entre los clústeres adyacentes. Para K=2, tenemos puntajes razonables, ya que el primer clúster alcanza 0.6, mientras que el segundo tiene un pico correspondiente a alrededor de 0.8. Sin embargo, aunque en este último la mayoría de las muestras se caracterizan por tener s(•) > 0.75, en el primero, aproximadamente la mitad de las muestras están por debajo de 0.5. Este análisis muestra que el clúster más grande es más homogéneo y es más fácil para K-means asignar las muestras (es decir, en términos de medidas, la varianza de xi ∈ K2 es menor y, en el espacio de alta dimensión, la esfera que representa a K2 es más uniforme que la que representa a K1)

Los demás gráficos muestran escenarios similares porque se ha detectado un clúster muy cohesivo junto con algunos clústeres agudos. Esto significa que hay una discrepancia de anchura muy consistente. Sin embargo, al aumentar K, obtenemos clústeres ligeramente más homogéneos porque tiende a ser similar el número de muestras asignadas. La presencia de un clúster muy redondeado (casi rectangular) con s(•) > 0.75 confirma que el conjunto de datos contiene al menos un grupo de muestras muy cohesivas, cuyas distancias con respecto a cualquier otro punto asignado a otros clústeres son bastante cercanas. Sabemos que la clase maligna (aunque su cardinalidad sea mayor) es más compacta, mientras que la benigna se extiende sobre un subespacio mucho más amplio; por lo tanto, podemos asumir que para todos los K, el clúster más redondeado está compuesto por muestras malignas y todos los demás se pueden distinguir según su nitidez. Por ejemplo, para K=8, es muy probable que el tercer clúster corresponda a la parte central del segundo clúster en el primer gráfico, mientras que los más pequeños contienen muestras que pertenecen a regiones aisladas del subconjunto benigno.

Si no conocemos la verdad fundamental, deberíamos considerar tanto K=2 como K=8 (o incluso un número mayor). De hecho, en el primer caso, probablemente estemos perdiendo muchos detalles finos de información, pero estamos determinando una subdivisión sólida (asumiendo que un clúster no es extremadamente cohesivo debido a la naturaleza del problema). Por otro lado, con K>8, los clústeres son obviamente más pequeños, con una cohesión moderadamente mayor, y representan subgrupos que comparten algunas características comunes. Como discutimos en la sección anterior, la elección final depende de muchos factores y estas herramientas solo pueden proporcionar una indicación general. Además, cuando los clústeres no son convexos o su varianza no está distribuida uniformemente entre todas las características, K-means siempre dará rendimientos subóptimos porque los clústeres resultantes incorporarán un espacio grande vacío. Sin direcciones específicas, el número óptimo de clústeres está asociado con un gráfico que contiene gráficos redondeados homogéneos (con aproximadamente el mismo ancho). Si la forma permanece afilada para cualquier valor de K, significa que la geometría no es completamente compatible con medidas simétricas (por ejemplo, los clústeres están muy estirados) y se deben considerar otros métodos.